In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 !pip install Pillow

In [ ]:
import os
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from torchvision.utils import save_image
from sklearn.datasets import fetch_lfw_people
from PIL import Image

def to_img(x):
    return x.view(x.size(0), 1, 50, 37)

def save_sample(img, name):
    if not os.path.exists('./samples'):
        os.makedirs('./samples')
    save_image(img.view(1, 50, 37), f'./samples/{name}.png')

def add_noise(img):
    return torch.clamp(img + torch.randn(img.size()) * 0.2, 0., 1.)

dataset = fetch_lfw_people(min_faces_per_person=70, resize=0.4).images / 255
dataset = torch.tensor(dataset, dtype=torch.float32).unsqueeze(1)
dataloader = DataLoader(TensorDataset(dataset), batch_size=4, shuffle=True)

class Autoencoder(nn.Module):
    def __init__(self): # Ganti _init_ menjadi __init__
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(50 * 37, 256),
            nn.ReLU(True),
            nn.Linear(256, 64)
        )
        self.decoder = nn.Sequential(
            nn.Linear(64, 256),
            nn.ReLU(True),
            nn.Linear(256, 50 * 37),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

cuda = torch.cuda.is_available()
model = Autoencoder().cuda() if cuda else Autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

for epoch in range(1, 51):
    for data in dataloader:
        img = data[0]
        noisy_img = add_noise(img)
        img, noisy_img = img.view(img.size(0), -1), noisy_img.view(noisy_img.size(0), -1)

        if cuda:
            img, noisy_img = img.cuda(), noisy_img.cuda()

        output = model(noisy_img)
        loss = criterion(output, img)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch}/50], Loss: {loss.item():.4f}')

    if epoch % 10 == 0:
        save_sample(to_img(img.cpu().data)[0], f'epoch_{epoch}_original')
        save_sample(to_img(output.cpu().data)[0], f'epoch_{epoch}_reconstructed')

print("Training selesai.")




Epoch [1/50], Loss: 0.0000
Epoch [2/50], Loss: 0.0000
Epoch [3/50], Loss: 0.0000
Epoch [4/50], Loss: 0.0000
Epoch [5/50], Loss: 0.0000
Epoch [6/50], Loss: 0.0000
Epoch [7/50], Loss: 0.0000
Epoch [8/50], Loss: 0.0000
Epoch [9/50], Loss: 0.0000
Epoch [10/50], Loss: 0.0000
Epoch [11/50], Loss: 0.0000
Epoch [12/50], Loss: 0.0000
Epoch [13/50], Loss: 0.0000
Epoch [14/50], Loss: 0.0000
Epoch [15/50], Loss: 0.0000
Epoch [16/50], Loss: 0.0000
Epoch [17/50], Loss: 0.0000
Epoch [18/50], Loss: 0.0000
Epoch [19/50], Loss: 0.0000
Epoch [20/50], Loss: 0.0000
Epoch [21/50], Loss: 0.0000
Epoch [22/50], Loss: 0.0000
Epoch [23/50], Loss: 0.0000
Epoch [24/50], Loss: 0.0000
Epoch [25/50], Loss: 0.0000
Epoch [26/50], Loss: 0.0000
Epoch [27/50], Loss: 0.0000
Epoch [28/50], Loss: 0.0000
Epoch [29/50], Loss: 0.0000
Epoch [30/50], Loss: 0.0000
Epoch [31/50], Loss: 0.0000
Epoch [32/50], Loss: 0.0000
Epoch [33/50], Loss: 0.0000
Epoch [34/50], Loss: 0.0000
Epoch [35/50], Loss: 0.0000
Epoch [36/50], Loss: 0.0000
E